In [ ]:
!pip install openmeteo-requests
!pip install requests-cache retry-requests numpy pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 6.6 MB/s eta 0:00:00


In [ ]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 37.615223,
	"longitude": -122.389977,
	"start_date": "2023-02-01",
	"end_date": "2024-05-18",
	"hourly": ["temperature_2m","relative_humidity_2m","precipitation","pressure_msl","surface_pressure","cloud_cover","wind_speed_10m"]
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
hourly_precipitation = hourly.Variables(2).ValuesAsNumpy()
hourly_pressure_msl = hourly.Variables(3).ValuesAsNumpy()
hourly_cloud_cover = hourly.Variables(4).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(5).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
hourly_data["precipitation"] = hourly_precipitation
hourly_data["pressure_msl"] = hourly_pressure_msl
hourly_data["cloud_cover"] = hourly_cloud_cover
hourly_data["wind_speed_10m"] = hourly_wind_speed_10m

hourly_dataframe_past = pd.DataFrame(data = hourly_data)
print(hourly_dataframe_past)

Coordinates 37.644989013671875°N -122.44325256347656°E
Elevation 25.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
                           date  temperature_2m  relative_humidity_2m  \
0     2023-02-01 00:00:00+00:00         12.3315             37.520824   
1     2023-02-01 01:00:00+00:00         10.3315             59.707893   
2     2023-02-01 02:00:00+00:00          8.4315             60.962593   
3     2023-02-01 03:00:00+00:00          7.7815             68.920547   
4     2023-02-01 04:00:00+00:00          7.2315             77.622498   
...                         ...             ...                   ...   
11347 2024-05-18 19:00:00+00:00             NaN                   NaN   
11348 2024-05-18 20:00:00+00:00             NaN                   NaN   
11349 2024-05-18 21:00:00+00:00             NaN                   NaN   
11350 2024-05-18 22:00:00+00:00             NaN                   NaN   
11351 2024-05-18 23:00:00+00:00             NaN                   NaN

In [ ]:
# Save the daily averages data to a new CSV file
output_file_path = 'hourly_dataframe_past.csv'  # Using a simple file name for easy download
hourly_dataframe_past.to_csv(output_file_path, index=False)

# Download the file in Google Colab
from google.colab import files
files.download(output_file_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": 37.615223,
	"longitude": -122.389977,
	"hourly": ["temperature_2m","relative_humidity_2m","precipitation","pressure_msl","surface_pressure","cloud_cover","wind_speed_10m"],
	"forecast_days": 16,
  "timezone": "UTC"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
hourly_precipitation = hourly.Variables(2).ValuesAsNumpy()
hourly_pressure_msl = hourly.Variables(3).ValuesAsNumpy()
hourly_cloud_cover = hourly.Variables(4).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(5).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
hourly_data["precipitation"] = hourly_precipitation
hourly_data["pressure_msl"] = hourly_pressure_msl
hourly_data["cloud_cover"] = hourly_cloud_cover
hourly_data["wind_speed_10m"] = hourly_wind_speed_10m


hourly_dataframe_forecast = pd.DataFrame(data = hourly_data)
print(hourly_dataframe_forecast)

Coordinates 37.62610626220703°N -122.40017700195312°E
Elevation 25.0 m asl
Timezone b'UTC' b'UTC'
Timezone difference to GMT+0 0 s
                         date  temperature_2m  relative_humidity_2m  \
0   2024-05-18 00:00:00+00:00       17.507000                  73.0   
1   2024-05-18 01:00:00+00:00       15.907000                  75.0   
2   2024-05-18 02:00:00+00:00       15.207001                  77.0   
3   2024-05-18 03:00:00+00:00       13.707001                  79.0   
4   2024-05-18 04:00:00+00:00       12.707001                  79.0   
..                        ...             ...                   ...   
379 2024-06-02 19:00:00+00:00             NaN                   NaN   
380 2024-06-02 20:00:00+00:00             NaN                   NaN   
381 2024-06-02 21:00:00+00:00             NaN                   NaN   
382 2024-06-02 22:00:00+00:00             NaN                   NaN   
383 2024-06-02 23:00:00+00:00             NaN                   NaN   

     precipitati

In [ ]:
# Save the daily averages data to a new CSV file
output_file_path = 'hourly_dataframe_forecast.csv'  # Using a simple file name for easy download
hourly_dataframe_forecast.to_csv(output_file_path, index=False)

# Download the file in Google Colab
from google.colab import files
files.download(output_file_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>